Create a scraper that searches a news site for keywords and adds them to a CSV file.


Keywords that I will search for:

1.   Medicine
2.   Breakthrough
3.   Upcoming
4.   Discovery



In [5]:
#First thing first as always, load in the packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

Next I am going to create a function that will grab the article and relevant information. For somereason it got stuck on one article so Gemini helped me rewrite the author to handle that.

In [7]:
#This function grabs the url and pulls out all of the relevant information
def grabarticle (thisurl):
  thispage = requests.get(thisurl)
  bs = BeautifulSoup(thispage.text, 'html.parser')
  paragraphs = bs.find('main').find_all('p')
  title = bs.find('h1').text
  author_tag = bs.find(itemprop='author')
  author = author_tag.text[2:].strip() if author_tag else 'No author found' #Gets the author, with help from gemini
  dateline = bs.find('time').text.strip() #Gets the date
  article_text = '\n\n'.join(p.text for p in paragraphs) # Finds the text of the article
  return {'url':thisurl, 'author':author, 'title':title,
          'dateline':dateline, 'article':article_text}

Next we build the scrapper.

In [9]:
thispage = requests.get('https://news.asu.edu/')
bs = BeautifulSoup(thispage.text, 'html.parser')


In [11]:
#Collecting the links from the main news page
links = bs.find('main').find_all('a')
link_list = []
for link in links: #The best way to do this was to filter for the main page to get just the articles
  if "https://news.asu.edu" in link.get('href'): #kept getting an error when I tried ['href'] so AI suggested link.get()
    link_list.append(link.get('href'))
print(link_list)


['https://news.asu.edu/20251022-science-and-technology-campus-company-how-research-sparked-new-water-tech-startup', 'https://news.asu.edu/20251106-health-and-medicine-life-service-how-asus-cheryl-schmidt-turned-duty-legacy', 'https://news.asu.edu/20251103-science-and-technology-smarter-fuel-pipelines-safer-communities', 'https://news.asu.edu/20251030-university-news-asu-ranked-among-world-best-graduate-employability', 'https://news.asu.edu/2025022-science-and-technology-asu-computer-scientists-ai-jobs-and-road-ahead', 'https://news.asu.edu/20250915-science-and-technology-asu-student-works-protect-african-penguins-local-aquarium', 'https://news.asu.edu/20251110-sun-devil-community-serving-those-who-are-built-serve', 'https://news.asu.edu/20251107-university-news-asu-state-arizona-launch-initiative-power-public-workforce', 'https://news.asu.edu/20251106-law-journalism-and-politics-first-amendment-academy-empowers-citizens-knowledge-five-freedoms', 'https://news.asu.edu/20251110-health-an

Now to build the actual scraper that will collect all of the information for the links.

In [13]:
#First initialize the dataframe
df = pd.DataFrame(columns=['url', 'author', 'title', 'dateline', 'article', 'keyword'])
keywords = ['medicine', 'breakthrough', 'upcoming', 'discovery']

In [15]:
#Now we are going to do the heavy lifting
for eachlink in link_list:
  story = grabarticle(eachlink)
  sleep(15)
  for keyword in keywords:
    if keyword in story['article']:
      story['keyword'] = keyword
      df = pd.concat([df, pd.DataFrame([story])], ignore_index=True) #pandas no longer has the .append method for dataframes, this is the workaround provided by AI
      print('getting', eachlink)


getting https://news.asu.edu/20251107-health-and-medicine-asu-researcher-importance-considering-sex-differences-health-studies
getting https://news.asu.edu/20251107-health-and-medicine-asu-researcher-importance-considering-sex-differences-health-studies
getting https://news.asu.edu/20251104-local-national-and-global-affairs-asu-taiwan-strengthen-partnership-ties
getting https://news.asu.edu/20251103-sun-devil-community-asu-hosts-welcome-home-tribute-vietnam-veterans-during-salute-service
getting https://news.asu.edu/20250807-environment-and-sustainability-preparing-drier-future-colorado-river-basin
getting https://news.asu.edu/20251110-science-and-technology-former-nsf-director-aims-help-make-science-tech-more-accessible
getting https://news.asu.edu/20251110-science-and-technology-former-nsf-director-aims-help-make-science-tech-more-accessible
getting https://news.asu.edu/20251107-health-and-medicine-asu-researcher-importance-considering-sex-differences-health-studies
getting https://n

In [17]:
df

,url,author,title,dateline,article,keyword
0,https://news.asu.edu/20251107-health-and-medic...,Kimberlee D’Ardenne,ASU researcher on the importance of considerin...,"November 07, 2025",President's Professor Heather Bimonte-Nelson s...,medicine
1,https://news.asu.edu/20251107-health-and-medic...,Kimberlee D’Ardenne,ASU researcher on the importance of considerin...,"November 07, 2025",President's Professor Heather Bimonte-Nelson s...,discovery
2,https://news.asu.edu/20251104-local-national-a...,Mary Beth Faller,"ASU, Taiwan strengthen partnership ties","November 04, 2025","President Crow talks TSMC workforce, academic ...",discovery
3,https://news.asu.edu/20251103-sun-devil-commun...,Jerry Gonzalez,ASU hosts 'welcome home' tribute for Vietnam v...,"November 03, 2025","Event to bring campus, local community togethe...",medicine
4,https://news.asu.edu/20250807-environment-and-...,Marshall Terrill,Preparing for a drier future on the Colorado R...,"August 07, 2025",With a looming deadline for the Colorado River...,breakthrough
5,https://news.asu.edu/20251110-science-and-tech...,Scott Bordow,"Former NSF director aims to help make science,...","November 10, 2025","'We, as a nation, can outcompete by out-innova...",medicine
6,https://news.asu.edu/20251110-science-and-tech...,Scott Bordow,"Former NSF director aims to help make science,...","November 10, 2025","'We, as a nation, can outcompete by out-innova...",discovery
7,https://news.asu.edu/20251107-health-and-medic...,Kimberlee D’Ardenne,ASU researcher on the importance of considerin...,"November 07, 2025",President's Professor Heather Bimonte-Nelson s...,medicine
8,https://news.asu.edu/20251107-health-and-medic...,Kimberlee D’Ardenne,ASU researcher on the importance of considerin...,"November 07, 2025",President's Professor Heather Bimonte-Nelson s...,discovery
9,https://news.asu.edu/20251104-local-national-a...,Mary Beth Faller,"ASU, Taiwan strengthen partnership ties","November 04, 2025","President Crow talks TSMC workforce, academic ...",discovery


Now we just need to save the dataframe to a CSV file.

In [24]:
df.to_csv('C://Users//abiga//OneDrive//articleSearches.csv')